# TEMPRO

In [1]:
### Initialize seed for reproducibility
import numpy as np
np.random.seed(0)
import math

### Data Wrangling and Plots
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 600
plt.rcParams['savefig.dpi'] = 600
import seaborn as sns


from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import keras

## 1) Load the ESM Pretrained Models

In [3]:
### note: loading the ESM 15 Billion parameters takes longer
import torch
import esm

# Load ESM-2 model
# esm.pretrained.esm2_t48_15B_UR50D() 
# esm.pretrained.esm2_t36_3B_UR50D() 
# esm.pretrained.esm2_t33_650M_UR50D()

#model, alphabet = esm.pretrained.esm2_t48_15B_UR50D()
#model, alphabet = esm.pretrained.esm2_t36_3B_UR50D()
model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
batch_converter = alphabet.get_batch_converter()
model.eval()  

ESM2(
  (embed_tokens): Embedding(33, 1280, padding_idx=1)
  (layers): ModuleList(
    (0-32): 33 x TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (rot_emb): RotaryEmbedding()
      )
      (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=1280, out_features=5120, bias=True)
      (fc2): Linear(in_features=5120, out_features=1280, bias=True)
      (final_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    )
  )
  (contact_head): ContactPredictionHead(
    (regression): Linear(in_features=660, out_features=1, bias=True)
    (activation): Sigmoid()
  )
  (emb_layer_norm_after): LayerNorm((1280,), eps=1

## 2) Load fasta files (it can include single to mulitple sequences)

### 2a) declare fasta function and input fasta filename

In [4]:
def read_fasta(fp):
        name, seq = None, []
        for line in fp:
            line = line.rstrip()
            if line.startswith(">"):
                if name: yield (name, ''.join(seq))
                name, seq = line, []
            else:
                seq.append(line)
        if name: yield (name, ''.join(seq))

# input your fasta
#fasta_filename = 'sdabs.fasta'
#fasta_filename = 'sample.fasta'
fasta_filename = 'experimentals.fasta'

data = []
with open(fasta_filename) as fp:
    for name, seq in read_fasta(fp):
        data.append((name, seq))
        
data2 = data[0:9150] # Making sure the dataset being looked at is divisible by chunk/batch size
data2

[('>4IDL_1|Chain A|Single Domain Antibody VHH A9|Lama glama (9844)',
  'MAKVQLQQSGGGAVQTGGSLKLTCLASGNTASIRAMGWYRRAPGKQREWVASLTTTGTADYGDFVKGRFTISRDNANNAATLQMDSLKPEDTAVYYCNADGRRFDGARWREYESWGQGTQVTISSAAALEHHHHHH'),
 ('>4TYU_1|Chains A, B|Single Domain Antibody|Lama glama (9844)',
  'GSHMEVQLVESGGGLVQAGDSLRLSCTASGRTFSRAVMGWFRQAPGKEREFVAAISAAPGTAYYAFYADSVRGRFSISADSAKNTVYLQMNSLKPEDTAVYYCAADLKMQVAAYMNQRSVDYWGQGTQVTVSS'),
 ('>4U05_1|Chains A, B|Single Domain Antibody|Lama glama (9844)',
  'GSHMEVQLVESGGGLVQAGDSLRLSCTASGRTFSRAVMGWFRQAPGKEREFVAAISAAPGTAYYAFYADSVRGRFSIAADSAKNTVYLQMNSLKPEDTAVYYCAADLKMQVAAYMNQRSVDYWGQGTQVTVSS'),
 ('>4W68_1|Chains A, B|Single domain antibody|Lama glama (9844)',
  'GSHMEVQLVESGGGLVQAGDSLRLSATASGRTFSRAVMGWFRQAPGKEREFVAAISAAPGTAYYAFYADSVRGRFSISADSAKNTVYLQMNSLKPEDTAVYYVAADLKMQVAAYMNQRSVDYWGQGTQVTVSS'),
 ('>4W70_1|Chain A|Single Domain Antibody|Lama glama (9844)',
  'MAEVQLVESGGGLVQAGDSLRLSATASGRTFSRAVMGWFRQAPGKEREFVAAISAAPGTAYYAFYADSVRGRFSISADSAKNTVYLQMNSLKPEDTAVYYVAADL

### 2b) extract their ESM embeddings

In [5]:
sequence_representations_list = []
chunk_size = 25
for i in range(0, len(data2), chunk_size):
    chunk = data2[i:i+chunk_size]
    print(i+chunk_size)
    batch_labels, batch_strs, batch_tokens = batch_converter(chunk)
    batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)

    # Extract per-residue representations (on CPU)
    with torch.no_grad():
    #    results = model(batch_tokens, repr_layers=[48], return_contacts=True) # 15B
    #token_representations = results["representations"][48]
    #    results = model(batch_tokens, repr_layers=[36], return_contacts=True) # 3B
    #token_representations = results["representations"][36]
        results = model(batch_tokens, repr_layers=[33], return_contacts=True) # 650M
    token_representations = results["representations"][33]
    
    
    # Generate per-sequence representations via averaging
    # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
    sequence_representations = []
    for i, tokens_len in enumerate(batch_lens):
        sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))

    sequence_representations_list.append(sequence_representations) # torch.stack(sequence_representations))

25


### 2c) assign as X for model prediction

In [6]:
flat_list = [item for sublist in sequence_representations_list for item in sublist]
X = torch.stack(flat_list, dim=0).cpu().detach().numpy()
X
### think twice before saving
#np.savetxt("nanobody_embeddings.csv", X, delimiter=",")

array([[-0.05619194, -0.00791028, -0.0454305 , ..., -0.1043366 ,
        -0.01007742,  0.08740339],
       [-0.07241704,  0.01124016, -0.0123387 , ..., -0.07190275,
        -0.03326014,  0.14270751],
       [-0.07218102,  0.01011745, -0.013458  , ..., -0.07263383,
        -0.03575287,  0.14528897],
       [-0.06507177,  0.00978963, -0.01847906, ..., -0.05832915,
        -0.02844086,  0.13671044],
       [-0.0672899 ,  0.00924716, -0.0324568 , ..., -0.0796625 ,
        -0.01490547,  0.10339514],
       [-0.07008886, -0.00545676, -0.02323345, ..., -0.09700638,
        -0.00512653,  0.08155268]], dtype=float32)

## 3) Load your model of choice:

In [7]:
model = keras.models.load_model("./saved_ANNmodels_1500epoch/ESM_650M.keras")
#model = keras.models.load_model("./saved_ANNmodels_1500epoch/ESM_3B.keras")
#model = keras.models.load_model("./saved_ANNmodels_1500epoch/ESM_15B.keras")

## 4) Predict Tm

In [8]:
model.predict(X)

1/1 [==============================] - 0s 249ms/step


array([[59.854244],
       [75.07941 ],
       [75.905106],
       [60.497295],
       [67.250626],
       [61.582012]], dtype=float32)

### excerpt from paper

In [14]:
#model = keras.models.load_model("./saved_ANNmodels_1500epoch/pchars_maestro.keras")
#model = keras.models.load_model("./saved_ANNmodels_1500epoch/af2.keras")
#model = keras.models.load_model("./saved_ANNmodels_1500epoch/nsp3.keras")
model = keras.models.load_model("./saved_ANNmodels_1500epoch/ESM_650M.keras")
#model = keras.models.load_model("./saved_ANNmodels_1500epoch/ESM_3B.keras")
#model = keras.models.load_model("./saved_ANNmodels_1500epoch/ESM_15B.keras")

#data = pd.read_csv("./tm_predictors/tm_dataset_nsp.csv")
#data = pd.read_csv("./tm_predictors/tm_dataset_af2.csv")
#data = pd.read_csv("./tm_predictors/tm_dataset_pchars_maestro.csv")
data = pd.read_csv("./tm_predictors/tm_dataset_ESM_650M.csv", header=None)
#data = pd.read_csv("./tm_predictors/tm_dataset_ESM_3B.csv", header=None)
#data = pd.read_csv("./tm_predictors/tm_dataset_ESM_15B.csv", header=None)

y = pd.read_excel("./tm_predictors/sdab_data.xlsx")
y = y.tm
x = data

# assign 80:20 ratio
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

mae = mean_absolute_error(y_test, model.predict(x_test))
rmse = math.sqrt(mean_squared_error(y_test, model.predict(x_test)))
coeff_det=r2_score(y_test, model.predict(x_test))

print(mae, rmse, coeff_det)

# results should be the same as the paper: mae=4.03, rmse=5.66, r2=0.71



4/4 [==============================] - 0s 2ms/step
5.35292833981548 7.208072276728451 0.5359914055902835
